In [18]:
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import numpy as np

In [19]:
PATH_TO_DATASETS = os.path.join(os.getcwd(), "..", "Datasets")
PATH_TO_DATASETS = os.path.abspath(PATH_TO_DATASETS)

In [ ]:
def parse_pascal_xml(xml_path):
    root = ET.parse(xml_path).getroot()
    objs = []
    for obj in root.findall('object'):
        name = obj.findtext('name', default='object')
        bnd = obj.find('bndbox')
        if bnd is not None:
            xmin = int(float(bnd.findtext('xmin')))
            ymin = int(float(bnd.findtext('ymin')))
            xmax = int(float(bnd.findtext('xmax')))
            ymax = int(float(bnd.findtext('ymax')))
            objs.append({'type':'bndbox','xmin':xmin,'ymin':ymin,'xmax':xmax,'ymax':ymax,'name':name})
            continue
        poly = obj.find('polygon') or obj.find('segmentation') or obj.find('points')
        if poly is not None:
            pts = []
            for pt in poly.findall('pt'):
                x = int(float(pt.findtext('x'))); y = int(float(pt.findtext('y')))
                pts.append((x,y))
            if not pts:
                xs = [p.text for p in poly.findall('x')]
                ys = [p.text for p in poly.findall('y')]
                if xs and ys and len(xs)==len(ys):
                    pts = [(int(float(x)), int(float(y))) for x,y in zip(xs,ys)]
                else:
                    txt = poly.text
                    if txt:
                        for pair in txt.strip().split():
                            px,py = pair.strip().split(',')
                            pts.append((int(float(px)), int(float(py))))
            if pts:
                objs.append({'type':'polygon','points':pts,'name':name})
                continue
    return objs

def create_mask_for_class(image_size, annots, class_name):
    H, W = image_size
    mask = Image.new('L', (W, H), 0)
    draw = ImageDraw.Draw(mask)
    for a in annots:
        if a.get('name') != class_name:
            continue
        if a['type'] == 'bndbox':
            draw.rectangle([a['xmin'], a['ymin'], a['xmax'], a['ymax']], fill=1)
        elif a['type'] == 'polygon':
            draw.polygon(a['points'], fill=1)
    return np.array(mask, dtype=np.uint8)

def collect_classes_from_xmls(xml_dir):
    classes = set()
    if not os.path.exists(xml_dir):
        return []
    for fname in os.listdir(xml_dir):
        if not fname.lower().endswith('.xml'):
            continue
        xml_path = os.path.join(xml_dir, fname)
        try:
            objs = parse_pascal_xml(xml_path)
        except Exception as e:
            print(f"Warning: failed to parse {xml_path}: {e}")
            continue
        for o in objs:
            classes.add(o.get('name','object'))
    return sorted(classes)

In [ ]:
def generate_masks_for_dataset(current_dataset: str, dataset_split_type: str, background_mask: bool = True):
    images_dir = os.path.join(PATH_TO_DATASETS, current_dataset, f"{dataset_split_type}_images")
    xml_dir = os.path.join(PATH_TO_DATASETS, current_dataset, f"{dataset_split_type}_annots")
    masks_out_root = os.path.join(PATH_TO_DATASETS, current_dataset, f"{dataset_split_type}_masks")

    classes = collect_classes_from_xmls(xml_dir)
    for c in classes:
        os.makedirs(os.path.join(masks_out_root, c), exist_ok=True)

    for fname in sorted(os.listdir(images_dir)):
        if not fname.lower().endswith(('.png','.jpg','.jpeg','tif', '.tiff')):
            continue
        base = os.path.splitext(fname)[0]
        img_path = os.path.join(images_dir, fname)
        xml_path = os.path.join(xml_dir, base + '.xml')

        img = Image.open(img_path).convert('RGB')

        W, H = img.size
        annots = parse_pascal_xml(xml_path)

        present_classes = {a['name'] for a in annots if 'name' in a}
        for cls in present_classes:
            mask_np = create_mask_for_class((H, W), annots, cls)
            if mask_np.sum() == 0:
                continue
            if background_mask:
                mask_np = 1 - mask_np
            out_fname = f"{base}.png"
            out_path = os.path.join(masks_out_root, cls, out_fname)
            out_img = (mask_np * 255).astype(np.uint8)
            Image.fromarray(out_img).save(out_path)

    print(f"Masks written to: {masks_out_root}. Classes: {len(classes)}")

In [22]:
generate_masks_for_dataset(current_dataset="UAV 1", dataset_split_type="train", background_mask=True)
generate_masks_for_dataset(current_dataset="UAV 1", dataset_split_type="test", background_mask=True)
generate_masks_for_dataset(current_dataset="UAV 1", dataset_split_type="valid", background_mask=True)

Done. Masks written to: c:\Users\jokub\Desktop\Tiriamasis grupinis\NesvarbuSefoSpecialus\Datasets\UAV 1\train_masks. Classes: 10
Done. Masks written to: c:\Users\jokub\Desktop\Tiriamasis grupinis\NesvarbuSefoSpecialus\Datasets\UAV 1\test_masks. Classes: 9
Done. Masks written to: c:\Users\jokub\Desktop\Tiriamasis grupinis\NesvarbuSefoSpecialus\Datasets\UAV 1\valid_masks. Classes: 10
